In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json
import re

#Firstly, load the .json file to a DataFrame
with open('transaction-data-adhoc-analysis.json') as f:
    data = pd.read_json(f)

#Function to split multiple transactions by one buyer
def item_split_to_list(transaction_item):
    if ';' in transaction_item:
        item_list = transaction_item.split(';')
        return item_list
    else:
        return [transaction_item]
    
#Function to split the unique items into 'brand | item', and 'qty'
def brand_item(unique_item):
    item_detail_list = unique_item.split(',')
    return (item_detail_list[0]) + ' | ' + item_detail_list[1]
def qty(unique_item):
    item_detail_list = unique_item.split(',')
    return item_detail_list[2]

#Vectorize new function into the 'data' DataFrame (make a new column to preserve the 'transaction_items' col.)
data['transaction_item_list'] = data.apply(lambda x: item_split_to_list(x['transaction_items']), axis=1)

#Use .explode() function to expand the list into rows of the same index (will become a numpy array)
transaction_items_exploded = data['transaction_item_list'].explode()

#Create a new DF to store the numpy array
transaction_items_df = pd.DataFrame()

#Vectorize the 'brand | item' and 'qty' functions
vfunc_brand = np.vectorize(brand_item)
vfunc_qty = np.vectorize(qty)

#Dump the data into the 'transaction_items_df' using the vectorized functions
transaction_items_df['brand | item'] = vfunc_brand(transaction_items_exploded)
transaction_items_df['qty'] = vfunc_qty(transaction_items_exploded)

#Remove unwanted characters in the qty column (only gets the number part)
transaction_items_df['qty'] = transaction_items_df['qty'].str.replace(r'[^0-9]', '',regex=True)

#Convert 'qty' column into integers
transaction_items_df["qty"] = transaction_items_df["qty"].astype(str).astype(int)

#Create a new DataFrame to store unique store items w/o quantity getting in the way
inventory_df = pd.DataFrame(transaction_items_df['brand | item'].unique(), columns=['brand | item'])

#Get the quantity for each item
inventory_list = list(inventory_df['brand | item'])
total_qty = []
for i in range(len(inventory_list)):
    total_qty_item = 0
    for j in range(len(transaction_items_df)):
        if inventory_list[i] == transaction_items_df.iloc[j,0]:
            total_qty_item += transaction_items_df.iloc[j,1]
    total_qty.append(total_qty_item)

#Add the 'total_qty' list to the 'inventory_df' DataFrame
inventory_df['total qty']=total_qty
inventory_df

,brand | item,total qty
0,Exotic Extras | Beef Chicharon,59302
1,HealthyKid 3+ | Nutrional Milk,58728
2,Candy City | Orange Beans,59406
3,HealthyKid 3+ | Gummy Vitamins,59576
4,HealthyKid 3+ | Yummy Vegetables,59429
5,Candy City | Gummy Worms,59319
6,Exotic Extras | Kimchi and Seaweed,59390


In [2]:
#Identify the unit price of each item
dummy_df = pd.DataFrame()
def filter_single_items(trans_item):
    if ';' not in trans_item and '(x1)' in trans_item:
        return trans_item
dummy_df['brand | item']=data.apply(lambda x:filter_single_items(x['transaction_items']),axis=1)
def filter_unit_price(trans_item, value):
    if ';' not in trans_item and '(x1)' in trans_item:
        return value
dummy_df['unit price']=data.apply(lambda x:filter_unit_price(x['transaction_items'],x['transaction_value']),axis=1)
dummy_df.dropna(inplace=True)
dummy_df = dummy_df.drop_duplicates(keep='first')
dummy_df['brand | item'] = dummy_df.apply(lambda x:brand_item(x['brand | item']),axis=1)
dummy_df

,brand | item,unit price
3,HealthyKid 3+ | Yummy Vegetables,500.0
8,Candy City | Gummy Worms,150.0
55,Exotic Extras | Beef Chicharon,1299.0
61,Exotic Extras | Kimchi and Seaweed,799.0
142,Candy City | Orange Beans,199.0
204,HealthyKid 3+ | Gummy Vitamins,1500.0
227,HealthyKid 3+ | Nutrional Milk,1990.0


In [3]:
inventory_df

,brand | item,total qty
0,Exotic Extras | Beef Chicharon,59302
1,HealthyKid 3+ | Nutrional Milk,58728
2,Candy City | Orange Beans,59406
3,HealthyKid 3+ | Gummy Vitamins,59576
4,HealthyKid 3+ | Yummy Vegetables,59429
5,Candy City | Gummy Worms,59319
6,Exotic Extras | Kimchi and Seaweed,59390


In [4]:
#Using SQL .join method, we are able to join the two columns based on their unique values
inventory_df = pd.merge(inventory_df,dummy_df, how='inner')
inventory_df

,brand | item,total qty,unit price
0,Exotic Extras | Beef Chicharon,59302,1299.0
1,HealthyKid 3+ | Nutrional Milk,58728,1990.0
2,Candy City | Orange Beans,59406,199.0
3,HealthyKid 3+ | Gummy Vitamins,59576,1500.0
4,HealthyKid 3+ | Yummy Vegetables,59429,500.0
5,Candy City | Gummy Worms,59319,150.0
6,Exotic Extras | Kimchi and Seaweed,59390,799.0


In [5]:
#Find the total sale
def total_sale_per_item(qty,unit_price):
    return qty * unit_price

inventory_df['total sale']=inventory_df.apply(lambda x:total_sale_per_item(x['total qty'],x['unit price']), axis=1)
inventory_df

,brand | item,total qty,unit price,total sale
0,Exotic Extras | Beef Chicharon,59302,1299.0,77033298.0
1,HealthyKid 3+ | Nutrional Milk,58728,1990.0,116868720.0
2,Candy City | Orange Beans,59406,199.0,11821794.0
3,HealthyKid 3+ | Gummy Vitamins,59576,1500.0,89364000.0
4,HealthyKid 3+ | Yummy Vegetables,59429,500.0,29714500.0
5,Candy City | Gummy Worms,59319,150.0,8897850.0
6,Exotic Extras | Kimchi and Seaweed,59390,799.0,47452610.0


In [6]:
#Get the monthly sale for 6 months:
inventory_df_monthly_sale = pd.DataFrame()
inventory_df_monthly_sale['brand | item'] = inventory_df['brand | item']
inventory_df_monthly_sale['unit price'] = inventory_df['unit price']
inventory_df_monthly_sale

,brand | item,unit price
0,Exotic Extras | Beef Chicharon,1299.0
1,HealthyKid 3+ | Nutrional Milk,1990.0
2,Candy City | Orange Beans,199.0
3,HealthyKid 3+ | Gummy Vitamins,1500.0
4,HealthyKid 3+ | Yummy Vegetables,500.0
5,Candy City | Gummy Worms,150.0
6,Exotic Extras | Kimchi and Seaweed,799.0


In [7]:
def transaction_date_per_item(transaction_item_list,transaction_date):
    transaction_dates = []
    for element in transaction_item_list:
        transaction_dates.append(transaction_date)
    return transaction_dates

data['transaction_date_list'] = data.apply(lambda x:transaction_date_per_item(x['transaction_item_list'],x['transaction_date']),axis=1)

In [8]:
transaction_dates_exploded = data['transaction_date_list'].explode(ignore_index=True)
transaction_items_df['transaction_date'] = transaction_dates_exploded
transaction_items_df

,brand | item,qty,transaction_date
0,Exotic Extras | Beef Chicharon,4,2022/01/12
1,Exotic Extras | Beef Chicharon,3,2022/01/12
2,HealthyKid 3+ | Nutrional Milk,4,2022/01/12
3,Candy City | Orange Beans,1,2022/01/12
4,HealthyKid 3+ | Gummy Vitamins,3,2022/01/08
...,...,...,...
166114,HealthyKid 3+ | Nutrional Milk,2,2022/06/09
166115,Candy City | Gummy Worms,2,2022/06/09
166116,HealthyKid 3+ | Nutrional Milk,1,2022/06/03
166117,Candy City | Gummy Worms,3,2022/06/03


In [9]:
#Split the transaction_date to get the month value easier
transaction_items_df[['year', 'month', 'date']] = transaction_items_df['transaction_date'].str.split('/', expand=True)
transaction_items_df

,brand | item,qty,transaction_date,year,month,date
0,Exotic Extras | Beef Chicharon,4,2022/01/12,2022,01,12
1,Exotic Extras | Beef Chicharon,3,2022/01/12,2022,01,12
2,HealthyKid 3+ | Nutrional Milk,4,2022/01/12,2022,01,12
3,Candy City | Orange Beans,1,2022/01/12,2022,01,12
4,HealthyKid 3+ | Gummy Vitamins,3,2022/01/08,2022,01,08
...,...,...,...,...,...,...
166114,HealthyKid 3+ | Nutrional Milk,2,2022/06/09,2022,06,09
166115,Candy City | Gummy Worms,2,2022/06/09,2022,06,09
166116,HealthyKid 3+ | Nutrional Milk,1,2022/06/03,2022,06,03
166117,Candy City | Gummy Worms,3,2022/06/03,2022,06,03


In [10]:
inventory_list = list(inventory_df['brand | item'])
#Month list can be edited accordingly
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June']
#Counts the number of items sold per month
for i in range(len(months)):
    total_qty_monthly_list = []
    for x in range(len(inventory_list)):
        totay_qty_item_month = 0
        for j in range(len(transaction_items_df)):
            if inventory_list[x] == transaction_items_df.iloc[j,0] and f'0{i+1}'==transaction_items_df.iloc[j,4]:
                totay_qty_item_month += transaction_items_df.iloc[j,1]
        total_qty_monthly_list.append(totay_qty_item_month)
    inventory_df_monthly_sale[f'{months[i]} qty']=total_qty_monthly_list
    inventory_df_monthly_sale[f'{months[i]} sales']=inventory_df_monthly_sale.apply(lambda x:total_sale_per_item(x[f'{months[i]} qty'],x['unit price']), axis=1)
        
inventory_df_monthly_sale

,brand | item,unit price,Jan qty,Jan sales,Feb qty,Feb sales,Mar qty,Mar sales,Apr qty,Apr sales,May qty,May sales,June qty,June sales
0,Exotic Extras | Beef Chicharon,1299.0,9665,12554835.0,10001,12991299.0,9816,12750984.0,9890,12847110.0,10028,13026372.0,9902,12862698.0
1,HealthyKid 3+ | Nutrional Milk,1990.0,9727,19356730.0,9691,19285090.0,9876,19653240.0,9786,19474140.0,9881,19663190.0,9767,19436330.0
2,Candy City | Orange Beans,199.0,9774,1945026.0,10037,1997363.0,9611,1912589.0,9914,1972886.0,9964,1982836.0,10106,2011094.0
3,HealthyKid 3+ | Gummy Vitamins,1500.0,9681,14521500.0,9980,14970000.0,10145,15217500.0,9842,14763000.0,9948,14922000.0,9980,14970000.0
4,HealthyKid 3+ | Yummy Vegetables,500.0,9959,4979500.0,10256,5128000.0,9896,4948000.0,9861,4930500.0,9735,4867500.0,9722,4861000.0
5,Candy City | Gummy Worms,150.0,9559,1433850.0,9996,1499400.0,9986,1497900.0,10043,1506450.0,9801,1470150.0,9934,1490100.0
6,Exotic Extras | Kimchi and Seaweed,799.0,9676,7731124.0,9949,7949251.0,9967,7963633.0,9921,7926879.0,9773,7808627.0,10104,8073096.0


In [26]:
qty_sold_table = pd.pivot_table(inventory_df_monthly_sale, values=['Jan qty','Feb qty','Mar qty','Apr qty','May qty','June qty'],index=['brand | item'], sort=False)
qty_sold_table

,Apr qty,Feb qty,Jan qty,June qty,Mar qty,May qty
brand | item,,,,,,
Exotic Extras | Beef Chicharon,9890,10001,9665,9902,9816,10028
HealthyKid 3+ | Nutrional Milk,9786,9691,9727,9767,9876,9881
Candy City | Orange Beans,9914,10037,9774,10106,9611,9964
HealthyKid 3+ | Gummy Vitamins,9842,9980,9681,9980,10145,9948
HealthyKid 3+ | Yummy Vegetables,9861,10256,9959,9722,9896,9735
Candy City | Gummy Worms,10043,9996,9559,9934,9986,9801
Exotic Extras | Kimchi and Seaweed,9921,9949,9676,10104,9967,9773


In [27]:
total_sales_table = pd.pivot_table(inventory_df_monthly_sale, values=['Jan sales','Feb sales','Mar sales','Apr sales','May sales','June sales'],index=['brand | item'], sort=False)
total_sales_table

,Apr sales,Feb sales,Jan sales,June sales,Mar sales,May sales
brand | item,,,,,,
Exotic Extras | Beef Chicharon,12847110.0,12991299.0,12554835.0,12862698.0,12750984.0,13026372.0
HealthyKid 3+ | Nutrional Milk,19474140.0,19285090.0,19356730.0,19436330.0,19653240.0,19663190.0
Candy City | Orange Beans,1972886.0,1997363.0,1945026.0,2011094.0,1912589.0,1982836.0
HealthyKid 3+ | Gummy Vitamins,14763000.0,14970000.0,14521500.0,14970000.0,15217500.0,14922000.0
HealthyKid 3+ | Yummy Vegetables,4930500.0,5128000.0,4979500.0,4861000.0,4948000.0,4867500.0
Candy City | Gummy Worms,1506450.0,1499400.0,1433850.0,1490100.0,1497900.0,1470150.0
Exotic Extras | Kimchi and Seaweed,7926879.0,7949251.0,7731124.0,8073096.0,7963633.0,7808627.0
